## Docker

```bash
sudo docker build -t maskrcnn-benchmark .
sudo docker run 
    --runtime=nvidia -d -it \
    --name=maskrcnn \
    -v=$(pwd):/notebooks \
    -p=8888:8888 \
    --ipc="host" \
    maskrcnn-benchmark
sudo docker logs maskrcnn
sudo docker start maskrcnn
sudo docker exec -it maskrcnn /bin/bash
```

## Notes

model-free vs model-based: model包含actor & critic，在critic方面，model-based有含renderer net, model-free沒有

In [19]:
# !pip install ray 
# ray[rllib] ray[debug] pandas

%cd /notebooks/LearningToPaint/rllib
!python canvas_env.py

/notebooks/LearningToPaint/rllib
W0831 19:14:11.173088 140287925675776 deprecation.py:323] From /miniconda/envs/py36/lib/python3.6/site-packages/tensorflow/python/compat/v2_compat.py:61: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
2019-08-31 19:14:11,744	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-31_19-14-11_743767_16823/logs.
2019-08-31 19:14:11,858	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:60571 to respond...
2019-08-31 19:14:12,001	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:62276 to respond...
2019-08-31 19:14:12,005	INFO services.py:809 -- Starting Redis shard with 1.67 GB max memory.
2019-08-31 19:14:12,050	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-31_19-14-11_743767_16823/logs

(pid=16852) 2019-08-31 19:14:21,010	INFO rollout_worker.py:742 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7fcee63ca438>}
(pid=16852) 2019-08-31 19:14:21,010	INFO rollout_worker.py:743 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.DictFlatteningPreprocessor object at 0x7fcee63d5dd8>}
(pid=16852) 2019-08-31 19:14:21,010	INFO rollout_worker.py:356 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7fcee63d5898>}
(pid=16852) 2019-08-31 19:14:21,070	INFO multi_gpu_optimizer.py:93 -- LocalMultiGPUOptimizer devices ['/cpu:0']
(pid=16852) Tensor("default_policy_1/tower/Reshape:0", shape=(?, 9, 9, 1), dtype=float32)
(pid=16852) Tensor("default_policy_1/tower_1/Reshape:0", shape=(?, 9, 9, 1), dtype=float32, device=/device:CPU:0)
(pid=16851) WARNING: Logging before flag parsing goes to stderr.
(pid=16851) W0831 19:14:25.146265 139986820921088 deprecation.py:323] From /minico

(pid=16851) 2019-08-31 19:14:33,903	INFO sampler.py:548 -- Outputs of compute_actions():
(pid=16851) 
(pid=16851) { 'default_policy': ( { 'data': { 'batches': [ np.ndarray((1,), dtype=int64, min=0.0, max=0.0, mean=0.0),
(pid=16851)                                                np.ndarray((1, 2), dtype=float32, min=0.089, max=1.518, mean=0.803)]},
(pid=16851)                         'type': 'TupleActions'},
(pid=16851)                       [],
(pid=16851)                       { 'action_prob': np.ndarray((1,), dtype=float32, min=0.051, max=0.051, mean=0.051),
(pid=16851)                         'behaviour_logits': np.ndarray((1, 5), dtype=float32, min=-0.019, max=0.037, mean=0.004),
(pid=16851)                         'vf_preds': np.ndarray((1,), dtype=float32, min=0.004, max=0.004, mean=0.004)})}
(pid=16851) 
(pid=16851) 2019-08-31 19:14:34,812	INFO sample_batch_builder.py:161 -- Trajectory fragment after postprocess_trajectory():
(pid=16851) 
(pid=16851) { 'agent0': { 'data': { 'act

(pid=16852) 2019-08-31 19:14:53,199	INFO multi_gpu_impl.py:146 -- Training on concatenated sample batches:
(pid=16852) 
(pid=16852) { 'inputs': [ np.ndarray((4000, 3), dtype=float32, min=-3.735, max=3.655, mean=-0.02),
(pid=16852)               np.ndarray((4000,), dtype=float32, min=-1146.852, max=0.0, mean=-57.455),
(pid=16852)               np.ndarray((4000, 166), dtype=float32, min=0.0, max=3.988, mean=0.174),
(pid=16852)               np.ndarray((4000, 3), dtype=float32, min=-3.735, max=3.655, mean=-0.02),
(pid=16852)               np.ndarray((4000,), dtype=float32, min=-1.982, max=2.208, mean=0.0),
(pid=16852)               np.ndarray((4000, 5), dtype=float32, min=-0.419, max=0.527, mean=-0.01),
(pid=16852)               np.ndarray((4000,), dtype=float32, min=-6157.924, max=-4.625, mean=-3247.304),
(pid=16852)               np.ndarray((4000,), dtype=float32, min=0.003, max=0.532, mean=0.073)],
(pid=16852)   'placeholders': [ <tf.Tensor 'default_policy/action:0' shape=(?, 3) dtype=

Traceback (most recent call last):
  File "canvas_env.py", line 180, in <module>
    "num_workers": 1,  # parallelism
  File "/miniconda/envs/py36/lib/python3.6/site-packages/ray/tune/tune.py", line 262, in run
    raise TuneError("Trials did not complete", errored_trials)
ray.tune.error.TuneError: ('Trials did not complete', [PPO_CanvasEnv_0])


In [8]:
# install openai baselines and other required packages
# !pip install gym tensorflow

# !apt-get update && apt-get -y install cmake libopenmpi-dev python3-dev zlib1g-dev
# %cd /notebooks/baselines
# # !git clone https://github.com/openai/baselines.git  # use github-desktop to clone repository
# !pip install -e .

# # install local gym-canvas as package
# %cd /notebooks/LearningToPaint/gym-canvas
# !pip install -e .

## Training

### train renderer

In [52]:
%cd /notebooks/LearningToPaint/pytorch-a2c-ppo-acktr-gail
!python main.py --env-name "gym_canvas:canvas-v0" --num-processes 1 --algo ppo --use-gae --lr 2.5e-4 --clip-param 0.1 --value-loss-coef 0.5 --num-steps 5 --num-mini-batch 4 --log-interval 1 --use-linear-lr-decay --entropy-coef 0.01

# !python baseline/train_renderer.py
# !tensorboard --logdir train_log --port=6006

/notebooks/LearningToPaint/pytorch-a2c-ppo-acktr-gail
Tuple
Traceback (most recent call last):
  File "main.py", line 254, in <module>
    main()
  File "main.py", line 55, in main
    base_kwargs={"recurrent": args.recurrent_policy},
  File "/notebooks/LearningToPaint/pytorch-a2c-ppo-acktr-gail/a2c_ppo_acktr/model.py", line 41, in __init__
    raise NotImplementedError
NotImplementedError


### train RL

* make data available

In [10]:
!python baseline/train.py --max_step=4 --env_batch=1 --warmup=1 --validate_interval=1 --debug

mkdir: cannot create directory './model': File exists
Renderer model loaded.
observation_space (1, 128, 128, 7) action_space 13
 #0: steps:4 interval_time:0.22 train_time:24.56
evaluating
 Step_0000007: mean_reward:0.288 mean_dist:0.201 var_dist:0.000
 #1: steps:8 interval_time:2.69 train_time:34.23
evaluating
 Step_0000011: mean_reward:-8.902 mean_dist:0.619 var_dist:0.000
 #2: steps:12 interval_time:2.55 train_time:43.24
evaluating
 Step_0000015: mean_reward:-7.753 mean_dist:0.417 var_dist:0.000
 #3: steps:16 interval_time:2.68 train_time:55.46
evaluating
 Step_0000019: mean_reward:-1.631 mean_dist:0.326 var_dist:0.000
^C


### Supervised training

In [1]:
!python baseline/train_supervised.py

Renderer model loaded.
step 0: loss 9167.975586
^C
Traceback (most recent call last):
  File "baseline/train_supervised.py", line 84, in <module>
    train(args.batch_size, args.episode_steps)